<a href="https://colab.research.google.com/github/clemgi0/movie-analyser_deep-learning-proyecto/blob/main/03_arquitectura_de_linea_de_base.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###DATASET
https://www.kaggle.com/datasets/harshitshankhdhar/imdb-dataset-of-top-1000-movies-and-tv-shows

In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import kagglehub
import os
import nltk
from nltk import word_tokenize
from tensorflow.keras.preprocessing.text import Tokenizer

In [45]:
path = kagglehub.dataset_download("harshitshankhdhar/imdb-dataset-of-top-1000-movies-and-tv-shows")

files_in_path = os.listdir(path)
csv_files = [f for f in files_in_path if f.endswith('.csv')]

if csv_files:
    data_file = os.path.join(path, csv_files[0])
    df = pd.read_csv(data_file)
    data = df.to_numpy()
    data = data[:, [1, 5, 7, 9, 6, 8]] # Name of the movie / Genre / Overview / Director / IMDB rating / meta-score
    print("Data shape:", data[:3,:])
else:
    print("No CSV files found in the specified path. Please specify which file to load if it's not a CSV or has a different extension.")

Using Colab cache for faster access to the 'imdb-dataset-of-top-1000-movies-and-tv-shows' dataset.
Data shape: [['The Shawshank Redemption' 'Drama'
  'Two imprisoned men bond over a number of years, finding solace and eventual redemption through acts of common decency.'
  'Frank Darabont' 9.3 80.0]
 ['The Godfather' 'Crime, Drama'
  "An organized crime dynasty's aging patriarch transfers control of his clandestine empire to his reluctant son."
  'Francis Ford Coppola' 9.2 100.0]
 ['The Dark Knight' 'Action, Crime, Drama'
  'When the menace known as the Joker wreaks havoc and chaos on the people of Gotham, Batman must accept one of the greatest psychological and physical tests of his ability to fight injustice.'
  'Christopher Nolan' 9.0 84.0]]


In [51]:
nltk.download('punkt_tab')
nltk.download('stopwords')
stopwords_en = nltk.corpus.stopwords.words('english')

# Remove the stopwords from nltk english stopwords dictionnary
cleaned_texts = np.array([])
for text in data[:,2]:
    tokens = [word.lower() for word in nltk.word_tokenize(text) if word.lower() not in stopwords_en]
    cleaned_texts = np.append(cleaned_texts, ' '.join(tokens))

# Tokenize the cleaned dataset of movie's resume
max_features = 5000
tokenizer = Tokenizer(num_words=max_features, split=' ')
tokenizer.fit_on_texts(cleaned_texts)
tokenizer.word_index.update({'<pad>': 0})
X_cleaned = tokenizer.texts_to_sequences(cleaned_texts)

# Convert each element of X_cleaned to a numpy array
X_cleaned = np.array([np.array(x) for x in X_cleaned], dtype=object)

# Retrieve the differents data's
x_train = np.column_stack((data[:800, [0, 1, 3]], X_cleaned[:800])) # Name of the movie / Genre / Overview / Director
y_train = data[:800, [4, 5]] # IMDB rating / meta-score

x_test = np.column_stack((data[800:, [0, 1, 3]], X_cleaned[800:])) # Name of the movie / Genre / Overview / Director
y_test = data[800:, [4, 5]] # IMDB rating / meta-score
print("Shapes of x_train and x_test:", x_train.shape, x_test.shape)
print("\n3 firsts samples of x_train:", x_train[:3,:])
print("\n3 firsts samples of x_test:", x_test[:3,:])
print("\nShapes of y_train and y_test:", y_train.shape, y_test.shape)
print("\n3 firsts samples of y_train:", y_train[:3,:])
print("\n3 firsts samples of y_test:", y_test[:3,:])

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Shapes of x_train and x_test: (800, 4) (200, 4)

3 firsts samples of x_train: [['The Shawshank Redemption' 'Drama' 'Frank Darabont'
  array([   5,  271,   48,  135, 1312,   25,  508, 1313, 2219,  272, 1314,
          921, 2220])                                                     ]
 ['The Godfather' 'Crime, Drama' 'Francis Ford Coppola'
  array([2221,   45, 2222,    1,  273, 1315, 2223,  922, 2224,  197,  923,
           26])                                                           ]
 ['The Dark Knight' 'Action, Crime, Drama' 'Christopher Nolan'
  array([2225,  413,  924, 1316, 1317, 1318,   46,  659,  660,   16, 2226,
           13,  925, 1319,  414, 2227, 2228,   82, 2229])                 ]]

3 firsts samples of x_test: [['Office Space' 'Comedy' 'Mike Judge'
  array([  30,  448, 2177, 1162, 2046,  433,  470, 1766,  410])]
 ['Happiness' 'Comedy, Drama' 'Todd Solondz'
  array([  32,  172, 1426, 1327,   90,   32,  900,  808, 1601, 1314,  242,
         1152,  910,   14, 4966,  219,   5